In [1]:
import nltk
import json
import random

## Carga de los datos

Punto de partida: un .json con una lista de proyectos asignados 

In [2]:
with open('../tmp/devex-corpus.json') as corpus_file:
    corpus = json.loads(corpus_file.read())

In [3]:
print(type(corpus))
print(corpus[0])

<class 'list'>
["GE-Tbilisi: Legislative impact assessment, drafting and representationService contract award noticeGeorgia1.Publication reference:EuropeAid/135035/DH/SER/GE.2.Publication date of the contract notice:30.4.2014.3.Lot number and lot title:1 lot only.4.Contract number and value:ENPI/2015/357121, 1 713 780 EUR.5.Date of award of the contract:23.1.2015.6.Number of tenders received:6.7.Overall score of chosen tender:99,54.8.Name, address and nationality of successful tenderer:IBF International Consulting SA, avenue Louise 209A, 1050 Brussels, BELGIUM.Center for International Legal Cooperation (CILC), Koninginnegracht 7, 2514 AA The Hague, NETHERLANDS.Deutsche Stiftung fűr internationale rechtliche Zusammenarbeit e.V. (IRZ), Ubierstraße 92, 53173 Bonn, GERMANY.ISO code of country: BE.9.Duration of contract:30 months.10.Contracting authority:European Union, represented by the European Commission, on behalf of and for the account of the partner country, Tbilisi, GEORGIA.11.Legal

## Preparado de los datos

En primer lugar, vamos a convertir las descripciones en listas de palabras.

In [4]:
descriptions = []
for description, company in corpus:
    words = description.split()
    descriptions.append((words, company))

Definimos dos funciones importantes: **word_filter** y **word_normalizer**:
- **word_filter**: toma una serie de palabras y devuelve solo las que consideramos útiles para clasificar.
- **word_normalizer**: normaliza las palabras (eg. lowercase, stemming/lemmatization (quedarse con la raíz), etc.)

In [5]:
def words_filter(words):
    return [w for w in words if len(w) > 4]

In [6]:
def words_normalizer(words):
    normalizer_functions = []
    # words to lowercase
    normalizer_functions.append(lambda w: w.lower())
    # remove all this signs of punctuation
    #for sign in ['.', ',', ';', ':', '!', '?', ')', '(', '[', ']']:
    for sign in ['.', ',', ';', ':', '!']:
        fn = lambda w, sign=sign: fn(w.replace(sign, '')) if sign in w else w
        normalizer_functions.append(fn)
    for fn in normalizer_functions:
        words = map(fn, words)
    return list(words)

A partir de las descripciones, tenemos que extraer una lista de palabras características ("word features"). Será simplemente la lista total de palabras que aparecen en nuestras descripciones. Usamos las siguientes funciones:

In [7]:
def get_words_in_descriptions(descriptions, words_normalizer, words_filter):
    """
    Given a list of descriptions [([description words], "company name")], join all the
    words into a single list and return it.
    """
    all_words = []
    for (words, company) in descriptions:
      all_words.extend(words_normalizer(words_filter(words)))
    return all_words

In [8]:
def get_word_features(wordlist):
    """Given a list of all the words, return a (dict_keys) list of unique words."""
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

(Una **FreqDist** es simplemente una cuenta de cuántas veces aparece cada palabra)

In [9]:
wordlist = nltk.FreqDist(get_words_in_descriptions(descriptions, words_normalizer, words_filter))
wordlist.most_common(10)

[('modification', 2174),
 ('duration', 1718),
 ('location', 1551),
 ('action', 1491),
 ('contract', 1434),
 ('theme', 1288),
 ('assistance', 931),
 ('development', 894),
 ('services', 877),
 ('management', 841)]

In [10]:
word_features = get_word_features(get_words_in_descriptions(descriptions, words_normalizer, words_filter))

## Creación del clasificador

Nuestro clasificador será una caja negra. Le proporcionaremos una descripción, y nos devolverá una clasificación (la empresa que más probable ganaría este contrato).

Nuestro clasificador necesitará un training set. Usaremos las primeras 500 descripciones como training set, y el resto como testing set.

In [11]:
# es importante tener grupos de training y testing homogéneos (mismas proporciones de cada empresa)
random.seed(54965)
random.shuffle(descriptions)

In [12]:
training_set = descriptions[:500]
testing_set = descriptions[500:]

Para el clasificador no podemos usar nuestras descripciones directamente. Necesitamos una función (*feature extractor*) que sea capaz de extraer las features de la descripción. Esta función solo toma la descripción y devuelve nuestras *word features* indicando si son relevantes (**aproximación más básica: si aparecen o no en la descripción**).

In [13]:
def extract_features(description):
    """
    Feature extractor. Given a description (as a list of words), returns our list of word features,
    indicating whether each word is relevant for the description.
    """
    # remove duplicates
    description_words = set(description)
    features = {}
    # word_features is defined outside
    for word in word_features:
        features['contains({})'.format(word)] = (word in description_words)
    return features

In [14]:
mod_extract_features = lambda description: extract_features(words_normalizer(words_filter(description)))

In [15]:
mod_extract_features('this is a project made in indonesia'.split())

{'contains(150000)': False,
 'contains(?attn)': False,
 'contains(niras)': False,
 'contains(realised)': False,
 'contains(item(new)': False,
 'contains(2008)': False,
 'contains(recommendations)': False,
 'contains(inhabitants)': False,
 'contains(3lot)': False,
 'contains(27749535)': False,
 'contains(broader)': False,
 'contains(-fire)': False,
 'contains(nanggroe)': False,
 'contains(recovering)': False,
 'contains(sub-commit)': False,
 'contains(betania)': False,
 'contains(besarabeasca)': False,
 'contains(peaceful)': False,
 'contains(http//wwweiborg/about/procurement/calls/hl-1124htm)': False,
 'contains(senior)': False,
 'contains(akademie)': False,
 'contains(ocnitsa)': False,
 'contains(potable)': False,
 'contains(services?)': False,
 'contains(destination)': False,
 'contains(starts)': False,
 'contains(fy10/12)': False,
 'contains(aid-169-c-00-10-00102)': False,
 'contains(ensemble)': False,
 'contains(agro-input)': False,
 'contains(borrower)': False,
 'contains(denkalia

Usamos esta función para crear el training set, convirtiendo las descripciones en listas de features entendibles por el clasificador:

In [16]:
training_set = nltk.classify.apply_features(mod_extract_features, training_set)
testing_set = nltk.classify.apply_features(mod_extract_features, testing_set)

finalmente, creamos el clasificador, usando nuestro training set para entrenarlo

In [17]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [18]:
classifier.show_most_informative_features()

Most Informative Features
     contains(financing) = True             UNDP : IBF In =     30.0 : 1.0
      contains(duration) = False          Chemon : AGRECO =     29.0 : 1.0
          contains(from) = True           Chemon : Ernst  =     29.0 : 1.0
contains(productorservicecode) = True           Chemon : Ernst  =     29.0 : 1.0
  contains(modification) = True           Chemon : Ernst  =     29.0 : 1.0
       contains(funding) = True           Chemon : Ernst  =     29.0 : 1.0
contains(contractactiontype) = True           Chemon : Ernst  =     29.0 : 1.0
contains(descriptionofcontractrequirement) = True           Chemon : Ernst  =     29.0 : 1.0
       contains(psc_cat) = True           Chemon : Ernst  =     29.0 : 1.0
       contains(number0) = True           Chemon : Ernst  =     26.0 : 1.0


## Application of the classifier

Ahora podemos probar el clasificador. Intentemos clasificar una sola descripción:

In [19]:
"El primer contrato ha sido asignado a la empresa {}".format(testing_set[0][1])

'El primer contrato ha sido asignado a la empresa Chemonics International Inc.'

Comprobemos si nuestro clasificador lo hace bien:

In [20]:
classifier.classify(testing_set[0][0])

'UNICEF'

¡Uhm! Pero, ¿qué hay del resto? Creamos una función para comprobar los aciertos en **todo** el training set:

In [21]:
def check_accuracy(training_set, classifier):
    """Given a training set and a classifier, apply it to all cases and return the percentage of hits."""
    hits = 0
    cases = 0
    for description, company in training_set:
        if classifier.classify(description) == company:
            hits += 1
        cases += 1
    return hits/cases

In [22]:
accuracy = check_accuracy(training_set, classifier)
print("Nuestro clasificador acierta en el {}% de los casos".format(accuracy*100))

Nuestro clasificador acierta en el 41.0% de los casos
